# Machine Learning LAB 2: SUPPORT VECTOR MACHINES

Course 2023/24: *M. Caligiuri*, *P. Talli*, *F. Lincetto*, *F. Chiariotti*, *P. Zanuttigh*

The notebook contains some simple tasks to be performed with **SUPPORT VECTOR MACHINES (SVM)**.

Complete all the **required code sections** and **answer to all the questions**.

### IMPORTANT for the evaluation score:

1. **Read carefully all cells** and **follow the instructions**.
2. **Re-run all the code from the beginning** to obtain the results for the final version of your notebook, since this is the way we will do it before evaluating your notebooks.
3. Make sure to fill the code in the appropriate places **without modifying the template**, otherwise you risk breaking later cells.
4. Please **submit the jupyter notebook file (.ipynb)**, do not submit python scripts (.py) or plain text files. **Make sure that it runs fine with the restat&run all command**.
5. **Answer the questions in the appropriate cells**, not in the ones where the question is presented.

## Weather Classification with Support Vector Machines

In this notebook we are going to explore the use of Support Vector Machines (SVM) for weather classification. We will use a dataset collected using the Luxottica **iSee** glasses. These devices provide multiple **sensors mounted inside the glasses**, which can be accessed through a bluetooth connection.

![I-SEE Glasses](data/isee.png "I-SEE")

The dataset corresponds to 8 hours of atmospherical data recordings sampled every 3 seconds.

The dataset labels are the following:

| ID  | Label       |
| :-: | :-:         |
| 0   | Sunny       |
| 1   | Rain        |
| 2   | Cloudy      |
| 3   | Mostly Clear|

---

## Preliminary step

Place your **name** and **ID number** (matricola) in the cell below. <br>
Also recall to **save the file as Surname_Name_LAB02.ipynb**, failure to do so will incur in a **lower grade**.

**Student name**: Pietro Bernardi

**ID Number**: 2097494

---

## Import all the necessary Python libraries

In [ ]:
%matplotlib inline

import numpy as np
import scipy as sp
import itertools
from matplotlib import pyplot as plt

import sklearn.metrics as skm
from sklearn.svm import SVC
from sklearn import linear_model

---

## Define the heplper functions

In this section you will find some helper functions (some already implemented, some to be implemented by you) that will be used in the following sections.
1. `load_dataset` -> to load the dataset from the file `data/lux.npz`,
2. `plot_input` -> to plot the input data,
3. `k_split` ->  to split the trainig dataset in k different folds,
4. `k_fold_cross_validation` -> to perform the k-fold cross validation.

**DO NOT CHANGE THE PRE-WRITTEN CODE UNLESS OTHERWISE SPECIFIED**

In [ ]:
# Helper function to load the dataset
def load_dataset(path: str) -> (np.ndarray, np.ndarray):
    with np.load(path) as data:
        x, y = data["x"], data["y"]
        
        # Normalize the data
        x -= x.mean(axis=0)
        x /= x.std(axis=0)
        
    return x, y

In [ ]:
# Function for plotting a image and printing the corresponding label
def plot_input(X_matrix: np.ndarray, labels: np.ndarray) -> None:
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    cmap = plt.cm.get_cmap('Accent', 4)
    im = ax.scatter(X_matrix[:,0], X_matrix[:,1], X_matrix[:,2], c=labels, cmap=cmap)
    im.set_clim(-0.5, 3.5)
    cbar=fig.colorbar(im, ticks=[0,1,2,3], orientation='vertical', cmap=cmap)
    cbar.ax.set_yticklabels(['Sunny', 'Rainy','Cloudy', 'Mostly clear']) 

In [ ]:
# Function to split the dataset in k different folds
def k_split(x: np.ndarray, y:np.ndarray, k: int, shuffle: bool = True) -> tuple[list[np.ndarray], list[np.ndarray]]:
    # Create a list of indices
    idx = np.arange(x.shape[0])
    # Shuffle the dataset
    if shuffle:
        # Randomly shuffle the indices
        np.random.shuffle(idx)
        # Shuffle the dataset
        x = x[idx]
        y = y[idx]

    # Split the dataset in k folds
    # ADD YOUR CODE HERE
    # if k does not exactly divide the dataset's numerosity, the first (idx_shape[0]%k) folds will have an
    # additional element.
    fold_sizes = np.full(k, idx.shape[0]//k, dtype=int)
    fold_sizes[:(idx.shape[0]%k)] += 1
    # computing the splits
    i = 0
    x_folds, y_folds = [], []
    for fs in fold_sizes:
        # getting the data's indexes for this fold
        i_upper = i + fs
        idxs_fold = idx[i:i_upper]
        # extracting the data
        x_folds.append(x[idxs_fold])
        y_folds.append(y[idxs_fold])
        # updating the lower index
        i = i_upper
    
    # returning the tuple
    return tuple((x_folds, y_folds))
    

In [ ]:
# Function to perform the k-fold cross validation
def k_fold_cross_validation(x_train: np.ndarray, y_train: np.ndarray, k: int, model: SVC, parameters: dict) -> tuple[tuple, tuple]:
    # Define the folds for the cross validation
    x_folds, y_folds = k_split(x_train, y_train, k)

    # Build a list containing all of the possible combinatioon of parameters
    params = list(itertools.product(*parameters.values()))

    # Initialize the dictionary of results
    results = {k: 0 for k in params}

    # For each param combination, perform the SVM training and testing
    for param in params:
        param = dict(zip(parameters.keys(), param))

        fold_accuracies = []
        
        # ADD YOUR CODE HERE
        # setting the model parameters
        model.set_params(**param)
        
        # for a given set of parameters
        # we have 'k' different splits where k-1 folds are used collectively as training set and the remaining one
        # is used as a test set in order to compute the fold accuracy
        for test_idx in range(k):
            # getting the indexes to index the training folds
            train_idxs = np.array([train_idx for train_idx in range(k) if train_idx != test_idx])
            # building the training and test sets for this split
            X_training_set = np.concatenate(np.array(x_folds)[train_idxs])
            X_test_set = x_folds[test_idx]
            y_training_set = np.concatenate(np.array(y_folds)[train_idxs])
            y_test_set = y_folds[test_idx]
            # fitting the model with the given set of parameters on this particular split
            model.fit(X_training_set, y_training_set)
            # scoring the model for accuracy
            fold_accuracies.append(model.score(X_test_set, y_test_set))
        
        # Compute the mean accuracy
        results[tuple(param.values())] = round(np.mean(fold_accuracies), 4)
    
    # Find the best parameters
    best_parameters = dict(zip(parameters.keys(), params[np.argmax(list(results.values()))]))
    best_accuracy = np.max(list(results.values()))
    best = (best_parameters, best_accuracy)

    # Add the param name to the results
    results = [({k: v for k, v in zip(parameters.keys(), p)}, a) for p, a in results.items()]

    return best, results

---

## A) Hyper-parameter search

### TO DO (A.0)

**Set** the random **seed** using your **ID**. If you need to change it for testing add a constant explicitly, eg.: 1234567 + 1

In [ ]:
# fix your ID ("numero di matricola") and the seed for random generator
# as usual you can try different seeds by adding a constant to the number:
# ID = 1234567 + X
ID = 2097494# YOUR ID
np.random.seed(ID)

Before proceding to the training steps, we **load the dataset and split it** in training and test set (while the **training** set is **typically larger**, here we set the number of training samples to 1000 and 4000 for the test data).
The **split** is **performed after applying a random permutation** to the dataset, such permutation will **depend on the seed** you set above.

In [ ]:
# Load the dataset using the helper function
X, y = load_dataset("data/lux.npz")
print(X.shape, y.shape)

In [ ]:
# The task is quite easy, let's add noise to make it more challenging!
# You can try without noise (comment the next 2 lines, easy task), with the suggested amount of noise,
# or play with the suggested amount of noise 

noise = np.random.normal(0, 0.1, X.shape)
X = X + noise

### TO DO (A.1)

**Divide** the **data into training and test set** (for this part use 1000 samples in the **first** set, 4000 in the **second** one). Make sure that each label is present at least 10 times in training. If it is not, then keep adding permutations to the initial data until this happens.


In [ ]:
# Random permute the data and split into training and test taking the first 1000
# data samples as training and 4000 samples as test
permutation = np.random.permutation(X.shape[0]) # ADD YOUR CODE HERE (replace None)

X = X[permutation] # ADD YOUR CODE HERE (replace None)
y = y[permutation] # ADD YOUR CODE HERE (replace None)

m_training = 1000
m_test = 4000

X_train = X[:m_training] # ADD YOUR CODE HERE (replace None)
X_test = X[m_training:(m_training+m_test)] # ADD YOUR CODE HERE (replace None)
y_train = y[:m_training] # ADD YOUR CODE HERE (replace None)
y_test = y[m_training:(m_training+m_test)] # ADD YOUR CODE HERE (replace None)

# checking if each label appears at least 10 times in training
perm_idx = m_training + m_test
l_freqs = np.unique(y_train, return_counts=True)[1]
while (np.sum(l_freqs >= 10))!=len(l_freqs):
    # need to add more permutations to the training dataset
    X_train = np.concatenate([X_train, [X[perm_idx]]])
    y_train = np.concatenate([y_train, [y[perm_idx]]])
    # re-computing
    l_freqs = np.unique(y_train, return_counts=True)[1]
    
print("X_train shape:", X_train.shape,"X_test shape:", X_test.shape,"||","y_train shape:",  y_train.shape,"y_test shape:", y_test.shape)

labels, freqs = np.unique(y_train, return_counts=True) # ADD YOUR CODE HERE. Hint: use np.unique() (replace None)
print("Labels in training dataset: ", labels)
print("Frequencies in training dataset: ", freqs)

In [ ]:
#Let's try the plotting function
plot_input(X_train,y_train)

### TO DO (A.2)

Use a SVM classfier with cross validation to pick a model. Use a 4-fold cross-validation. Let's start with a Linear kernel.

In [ ]:
# Parameters for linear SVM
parameters = {'C': [ 0.01, 0.1, 1, 10]}

# Define the model (without parameters)
svm = SVC(kernel='linear') # ADD YOUR CODE HERE (replace None)

# Perform the K-fold cross validation
best, results = k_fold_cross_validation(X_train, y_train, 4, svm, parameters) # ADD YOUR CODE HERE (replace None)

print ('RESULTS FOR LINEAR KERNEL')
print("="*25)
print("Best parameter set found:")
print("-"*25)
for k in best[0].keys():
    print(f"{k}\t%2.2f"%(best[0][k]))
print("-"*25)
print("Score with best parameter: %1.4f"%(best[1]))
print()
print("="*23)
print("All scores on the grid:")
print("-"*23)
out = ''
for pname in parameters.keys():
    out += pname+'\t'
out += 'SCORE\n'
for r in results:
    for pname in r[0]:
        out += ("%4.2f"%(r[0][pname]))+'\t'
    out += ("%1.4f"%(r[1]))+'\n\r'
print(out)

# saving the best results for later
best_params = {}
best_params['linear'] = best[0]

### TO DO (A.3)

Pick a model for the Polynomial kernel with degree=2.

In [ ]:
# Parameters for linear SVM
parameters = {'C': [0.01, 0.1, 1],'gamma':[0.01,0.1,1.]}

# Define an SVM with poly of degree 2 kernel (without parameters)
poly2_svm = SVC(kernel='poly', degree=2) # ADD YOUR CODE HERE (replace None)

# Perform the K-fold cross validation
best, results = k_fold_cross_validation(X_train, y_train, 4, poly2_svm, parameters) # ADD YOUR CODE HERE (replace None)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')
print("="*32)
print("Best parameter set found:")
print("-"*32)
for k in best[0].keys():
    print(f"{k}\t%2.2f"%(best[0][k]))
print("-"*32)

print("Score with best parameter: %1.4f"%(best[1]))
# ADD YOUR CODE HERE
print()
print("="*23)
print("All scores on the grid:")
print("-"*23)
out = ''
for pname in parameters.keys():
    out += pname+'\t'
out += 'SCORE\n'
for r in results:
    for pname in r[0]:
        out += ("%4.2f"%(r[0][pname]))+'\t'
    out += ("%1.4f"%(r[1]))+'\n\r'
print(out)

# saving the best results for later
best_params['poly2'] = best[0]

### TO DO (A.4)

Now let's try a higher degree for the polynomial kernel (e.g., 3rd degree).

In [ ]:
# Parameters for poly with higher degree kernel
parameters = {'C': [0.01, 0.1, 1],'gamma':[0.01, 0.1, 1]}

# Define an SVM with poly of higher degree kernel (without parameters)
degree = 3
poly_svm = SVC(kernel='poly', degree=degree) # ADD YOUR CODE HERE (replace None)

# Perform the K-fold cross validation
best, results = k_fold_cross_validation(X_train, y_train, 4, poly_svm, parameters) # ADD YOUR CODE HERE (replace None)

print (f"RESULTS FOR POLY DEGREE={degree} KERNEL")

print("="*32)
print("Best parameter set found:")
print("-"*32)
for k in best[0].keys():
    print(f"{k}\t%2.2f"%(best[0][k]))
print("-"*32)

print("Score with best parameter: %1.4f"%(best[1]))
# ADD YOUR CODE HERE
print()
print("="*23)
print("All scores on the grid:")
print("-"*23)
out = ''
for pname in parameters.keys():
    out += pname+'\t'
out += 'SCORE\n'
for r in results:
    for pname in r[0]:
        out += ("%4.2f"%(r[0][pname]))+'\t'
    out += ("%1.4f"%(r[1]))+'\n\r'
print(out)

# saving the best results for later
best_params['poly3'] = best[0]

### TO DO (A.5)

Pick a model for the Radial Basis Function kernel:

In [ ]:
# Parameters for rbf SVM
parameters = {'C': [0.1, 1, 10, 100],'gamma':[0.001, 0.01, 0.1,1]}

# Define an SVM with rbf kernel (without parameters)
rbf_svm = SVC(kernel='rbf') # ADD YOUR CODE HERE (replace None)

# Perform the K-fold cross validation
best, results = k_fold_cross_validation(X_train, y_train, 4, rbf_svm, parameters) # ADD YOUR CODE HERE (replace None)

print ('RESULTS FOR rbf KERNEL')

print("="*32)
print("Best parameter set found:")
print("-"*32)
for k in best[0].keys():
    print(f"{k}\t%2.2f"%(best[0][k]))
print("-"*32)

print("Score with best parameter: %1.4f"%(best[1]))
# ADD YOUR CODE HERE
print()
print("="*23)
print("All scores on the grid:")
print("-"*23)
out = ''
for pname in parameters.keys():
    out += pname+'\t'
out += 'SCORE\n'
for r in results:
    for pname in r[0]:
        out += ("%4.2f"%(r[0][pname]))+'\t'
    out += ("%1.4f"%(r[1]))+'\n\r'
print(out)

# saving the best results for later
best_params['rbf'] = best[0]

### TO DO (A.Q1) [Answer the following]

What do you observe when using RBF and polynomial kernels on this dataset ?

**ANSWER A.Q1:**: they seem to achieve higher score figures.

### TO DO (A.6)
Report here the best SVM kernel and parameters

In [ ]:
# Get training and test error for the best SVM model from CV
best_svm = SVC(kernel='rbf', C=100., gamma=0.1) # USE YOUR OPTIMAL PARAMETERS HERE (replace None)

# Run the svm model on the whole training set
best_svm.fit(X_train, y_train)
score_training = best_svm.score(X_train, y_train)
score_testing = best_svm.score(X_test, y_test)

# Compute the errors
# (error is 1 - svm.score)
training_error = 1.0 - score_training # ADD YOUR CODE (replace None)
test_error = 1.0 - score_testing # ADD YOUR CODE (replace None)

print ("Best SVM training error: %f" % training_error)
print ("Best SVM test error: %f" % test_error)

### TO DO (A.7)

Analyze how the gamma parameter (inversely proportional to standard deviation of Gaussian Kernel) impact the performances of the classifier

In [ ]:
# Test with different values of gamma
# use rbf kernel and C=1
gamma_rbf = SVC(kernel='rbf')
# Set gamma values
gamma_values = np.logspace(-5,2,8)
print(gamma_values)

In [ ]:
train_acc_list, test_acc_list = [], []

# ADD YOUR CODE TO TRAIN THE SVM MULTIPLE TIMES WITH THE DIFFERENT VALUES OF GAMMA
# PLACE THE TRAIN AND TEST ACCURACY FOR EACH TEST IN THE TRAIN AND TEST ACCURACY LISTS
for g in gamma_values:
    gamma_rbf.set_params(**{'C':1, 'gamma':g})
    gamma_rbf.fit(X_train, y_train)    
    train_acc_list.append(gamma_rbf.score(X_train, y_train))
    test_acc_list.append(gamma_rbf.score(X_test, y_test))

In [ ]:
# Plot
fig, ax = plt.subplots(1,2, figsize=(15,5))

ax[0].plot(gamma_values, train_acc_list)
ax[0].set_xscale('log')
ax[0].set_xlabel('gamma')
ax[0].set_ylabel('Train accuracy')
ax[0].grid(True)

ax[1].plot(gamma_values, test_acc_list)
ax[1].set_xscale('log')
ax[1].set_xlabel('gamma')
ax[1].set_ylabel('Test accuracy')
ax[1].grid(True)

plt.show()

## B) More data
Now let's do the same but using more data points for training.

### TO DO (B.0)

Choose a higher number of data points (e.g. x = 10000) for training data depending on your computing capability.

In [ ]:
X = X[permutation]
y = y[permutation]

m_training = 600 # ADD YOUR CODE: adjust depending on the capabilities of your PC (replace None)

X_train, X_test = X[:m_training], X[m_training:]
y_train, y_test = y[:m_training], y[m_training:]

labels, freqs = np.unique(y_train, return_counts=True) # ADD YOUR CODE (replace None)
print("Labels in training dataset: ", labels)
print("Frequencies in training dataset: ", freqs)

# initialize support variables for boundaries visualization
granularity = 25
x_max = np.abs(X).max()
x_range = np.linspace(-x_max, x_max, granularity)
x_grid = np.stack(np.meshgrid(x_range, x_range, x_range)).reshape(3, -1).T

### TO DO (B.1)

Let's try to use SVM with parameters obtained from the best model for $m_{training} =  10000$. Since it may take a long time to run, you can decide to just let it run for some time and stop it if it does not complete. If you decide to do this, report it in the TO DO (C.Q1) cell below.

In [ ]:
# Get training and test error for the best SVM model from CV
best_svm.fit(X_train, y_train)
train_score = best_svm.score(X_train, y_train)
test_score = best_svm.score(X_test, y_test)
training_error = 1.0 - train_score
test_error = 1.0 - test_score

print ("Best SVM training error: %f" % training_error)
print ("Best SVM test error: %f" % test_error)

## C) Boundaries Visualization

Now let us plot the classification boundaries. 

### TO DO (C.0)

Use the SVM to predict on the test set X_test. 

In [ ]:
rbf_svm_test = best_svm.predict(X_test) # ADD YOUR CODE (replace None)

We constructed a grid of all possible combinations of input values, we now use it to extract the classification boundaries of the three classifiers by having them predict on each input.

In [ ]:
rbf_SVM_grid = best_svm.predict(x_grid)

rbf_SVM_m = y_test == rbf_svm_test

fig = plt.figure(figsize=(20,36))
ax1 = fig.add_subplot(3, 1, 1, projection="3d", title="RBF")
ax2 = fig.add_subplot(3, 1, 2, projection="3d", title="POLY (degree=3)")
ax3 = fig.add_subplot(3, 1, 3, projection="3d", title="LINEAR")

ax1.scatter(x_grid[:,0], x_grid[:,1], x_grid[:,2], c=rbf_SVM_grid, linewidth=0, marker="s", alpha=.05,cmap='Accent')
ax1.scatter(X_test[rbf_SVM_m,0], X_test[rbf_SVM_m,1], X_test[rbf_SVM_m,2], c=y_test[rbf_SVM_m], linewidth=.5, edgecolor="k", marker=".",cmap='Accent')
ax1.scatter(X_test[~rbf_SVM_m,0], X_test[~rbf_SVM_m,1], X_test[~rbf_SVM_m,2], c=y_test[~rbf_SVM_m], linewidth=1, edgecolor="r", marker=".",cmap='Accent')
ax1.set_xlim([-x_max, x_max])
ax1.set_ylim([-x_max, x_max])
ax1.set_zlim([-x_max, x_max])

# using the polynomial kernel with the best parameters for it
best_poly_svm = SVC(kernel='poly', degree=3, C=best_params['poly3']['C'], gamma=best_params['poly3']['gamma'])
best_poly_svm.fit(X_train, y_train)
poly_SVM_grid = best_poly_svm.predict(x_grid)
poly_SVM_test = best_poly_svm.predict(X_test)
poly_SVM_m = y_test == poly_SVM_test
ax2.scatter(x_grid[:,0], x_grid[:,1], x_grid[:,2], c=poly_SVM_grid, linewidth=0, marker="s", alpha=.05,cmap='Accent')
ax2.scatter(X_test[poly_SVM_m,0], X_test[poly_SVM_m,1], X_test[poly_SVM_m,2], c=y_test[poly_SVM_m], linewidth=.5, edgecolor="k", marker=".",cmap='Accent')
ax2.scatter(X_test[~poly_SVM_m,0], X_test[~poly_SVM_m,1], X_test[~poly_SVM_m,2], c=y_test[~poly_SVM_m], linewidth=1, edgecolor="r", marker=".",cmap='Accent')
ax2.set_xlim([-x_max, x_max])
ax2.set_ylim([-x_max, x_max])
ax2.set_zlim([-x_max, x_max])

# using the linear kernel with the best parameters for it
best_linear_svm = SVC(kernel='linear', C=best_params['linear']['C'])
best_linear_svm.fit(X_train, y_train)
linear_SVM_grid = best_linear_svm.predict(x_grid)
linear_SVM_test = best_linear_svm.predict(X_test)
linear_SVM_m = y_test == linear_SVM_test
ax3.scatter(x_grid[:,0], x_grid[:,1], x_grid[:,2], c=linear_SVM_grid, linewidth=0, marker="s", alpha=.05,cmap='Accent')
ax3.scatter(X_test[linear_SVM_m,0], X_test[linear_SVM_m,1], X_test[linear_SVM_m,2], c=y_test[linear_SVM_m], linewidth=.5, edgecolor="k", marker=".",cmap='Accent')
ax3.scatter(X_test[~linear_SVM_m,0], X_test[~linear_SVM_m,1], X_test[~linear_SVM_m,2], c=y_test[~linear_SVM_m], linewidth=1, edgecolor="r", marker=".",cmap='Accent')
ax3.set_xlim([-x_max, x_max])
ax3.set_ylim([-x_max, x_max])
ax3.set_zlim([-x_max, x_max])

#ax2.view_init(elev=0, azim=0)

### TO DO (C.Q1) [Answer the following]**

Compare and discuss the results from SVM with m=600 and with m=10000 (or whatever value you set) training data points. If you stopped the SVM, include such aspect in your comparison.

**ANSWER C.Q1:** when a smaller training set (m=600) is used, the number of misclassified samples is of course greater than in the case of a bigger training set (m=10000). However, using the suggested amount of noise (0.1),  while with m=600 the training error is less than the test error, as it is expected, in the case with m=10000 the training error is larger than the test error. This could be an indication of overfitting.

### TO DO (C.1)

Plot the confusion matrix for the SVM classifier. The confusion matrix has one column for each predicted label and one row for each true label. 
It shows for each class in the corresponding row how many samples belonging to that class gets each possible output label. Notice that the diagonal contains the correctly classified samples, while the other cells correspond to errors. You can obtain it with the sklearn.metrics.confusion_matrix function (see the documentation). You can also print also the normalized confusion matrix.


In [ ]:
np.set_printoptions(precision=2, suppress=True) # for better aligned printing of confusion matrix use floatmode='fixed'

u, counts = np.unique(y_test, return_counts=True)
print("Labels and frequencies in test set: ", counts)

confusion_SVM = skm.confusion_matrix(y_test, rbf_svm_test)
print("\n Confusion matrix SVM  \n \n", confusion_SVM)
print("\n Confusion matrix SVM (normalized)   \n \n", confusion_SVM /counts[:,None] )

In [ ]:
fig = plt.figure()
    
im = plt.imshow(confusion_SVM /counts[:,None], cmap="Blues",interpolation='nearest')
plt.xticks([0,1,2,3], ['Sunny', 'Rainy','Cloudy', 'Mostly clear'],ha="right",rotation=30)
plt.yticks([0,1,2,3], ['Sunny', 'Rainy','Cloudy', 'Mostly clear'],ha="right",rotation=30)
cm = confusion_SVM /counts[:,None]
fmt = '.2f'
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], fmt),
        ha="center", va="center",
        color="white" if cm[i, j] > thresh else "black")

fig.tight_layout()
fig.colorbar(im, location='bottom')  
plt.show()

### TO DO (C.Q2) [Answer the following]

Have a look at the confusion matrix and comment on the obtained accuracies. Why some classes have lower accuracies and others an higher one? Make some guesses on the possible causes.

**ANSWER C.Q2:** the confusion matrix seems to suggest that the classification algorithm encounters difficulties when it comes to correctly classify cloudy and rainy situations. This in a way makes sense since cloudy and rainy situations exhibit similar atmospheric conditions. This behaviour is also dependent on the amount of noise being introduced in the data: the more noise is added, the more "confused" the cloudy/rainy classification gets.